In [111]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [112]:
data = pd.read_csv('data\gemstone.csv')

In [113]:
data.drop(labels=['id'], axis=1, inplace=True)

In [114]:
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [115]:
X = data.drop(labels=['price'], axis=1)

In [116]:
y = data[['price']]

In [117]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [118]:
y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


In [119]:
# categorical and numerical columns
categoricalColumns = X.select_dtypes(include='object').columns
numericalColumns = X.select_dtypes(exclude='object').columns
print(f"Categorical columns : {categoricalColumns}")
print(f"Numerical columns : {numericalColumns}")

Categorical columns : Index(['cut', 'color', 'clarity'], dtype='object')
Numerical columns : Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')


In [120]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [121]:
# numeriacl pipeline
num_pipeline = Pipeline(

    steps=[
        # Handling missing values
        ('imputer',SimpleImputer()),
        ('scaler', StandardScaler())
    ]
)

In [122]:
# categorical pipeline
cat_pipeline = Pipeline(

    steps=[
        # Handling missing values
        ('imputer',SimpleImputer(strategy='most_frequent')),  # Impute it by Mode
        # Encoding the categorical to interger data
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
)

In [123]:
# Transform the data
preprocessor = ColumnTransformer(
    [
        ('num_pipeline',num_pipeline,numericalColumns),
        ('cat_pipeline',cat_pipeline,categoricalColumns)
    ]
)

In [124]:
## Train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 30)

In [125]:
preprocessor.fit_transform(X_train)

array([[-0.97543926, -0.84960654, -0.12153081, ...,  4.        ,
         5.        ,  5.        ],
       [ 0.2351953 ,  1.83363716, -0.12153081, ...,  1.        ,
         1.        ,  2.        ],
       [ 0.49461699,  0.81585507,  0.39980029, ...,  3.        ,
         3.        ,  4.        ],
       ...,
       [ 0.45138004,  1.55606023, -0.6428619 , ...,  1.        ,
         3.        ,  2.        ],
       [ 0.66756478, -1.77486298,  1.44246248, ...,  4.        ,
         3.        ,  4.        ],
       [ 0.25681377,  0.81585507, -0.12153081, ...,  4.        ,
         3.        ,  2.        ]])

In [126]:
preprocessor.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [127]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [128]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,4.0,5.0,5.0
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,1.0,1.0,2.0
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,3.0,3.0,4.0
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,3.0,3.0,3.0
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,4.0,6.0,5.0


In [129]:
"""import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (4,4))

## plot the histogram chart for the numerical columns
for col in X_train.columns:
    sns.histplot(data = X_train , x = col, kde = True)
    print("\n")
    plt.show()"""

'import seaborn as sns\nimport matplotlib.pyplot as plt\n\nplt.figure(figsize = (4,4))\n\n## plot the histogram chart for the numerical columns\nfor col in X_train.columns:\n    sns.histplot(data = X_train , x = col, kde = True)\n    print("\n")\n    plt.show()'

In [130]:
"""Linear regression
Ridge regression
Lasso regression
Elastic net"""

'Linear regression\nRidge regression\nLasso regression\nElastic net'

In [131]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [132]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [133]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [134]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [135]:
models.items()

dict_items([('LinearRegression', LinearRegression()), ('Lasso', Lasso()), ('Ridge', Ridge()), ('Elasticnet', ElasticNet())])

In [136]:
models.keys()

dict_keys(['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet'])

In [137]:
models.values()

dict_values([LinearRegression(), Lasso(), Ridge(), ElasticNet()])

In [138]:
for modelItem in models.items():
    model = modelItem[1] # getting the value
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, mse, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(modelItem[0]) # getting the key
    model_list.append(modelItem[0])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("MSE:",mse)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1013.9047094344005
MAE: 674.0255115796833
MSE: 1028002.7598132562
R2 score 93.68908248567512


Lasso
Model Training Performance
RMSE: 1013.8723151049943
MAE: 675.2036880701621
MSE: 1027937.0713363609
R2 score 93.68948574778251


Ridge
Model Training Performance
RMSE: 1013.9058997761025
MAE: 674.0565132295895
MSE: 1028005.173600788
R2 score 93.68906766741333


Elasticnet
Model Training Performance
RMSE: 1513.9140350229761
MAE: 1050.7468664314322
MSE: 2291935.705439549
R2 score 85.92978759337907




In [139]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [140]:
r2_list

[0.9368908248567511,
 0.9368948574778251,
 0.9368906766741333,
 0.8592978759337908]